In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from dtaidistance import dtw, dtw_c, dtw_ndim
from numpy.random import choice, shuffle
from torch import nn

In [ ]:
%load_ext autoreload
%autoreload 1

from preprocessing import *
from models import Encoder, Decoder, Sequence2Sequence

%aimport preprocessing
%aimport models

# Example of multidimensional time-series

In [ ]:
# !mkdir -p data
# !wget https://www.cs.ucr.edu/%7Eeamonn/time_series_data_2018/UCRArchive_2018.zip ../data
# !unzip data/UCRArchive_2018.zip -d ../data

In [ ]:
# data_path = "../data/UCRArchive_2018/Earthquakes/Earthquakes_TRAIN.tsv"

In [ ]:
# data = pd.read_csv(data_path, header=None, delimiter="\t").values

# # to remove Nan (only first columns)
# data = data[:, ~np.isnan(data).any(0)]

# X = data[:, 1:]
# y = data[:, 0].astype(np.short)

# Accelerometer dataset 

- 1: Working at Computer
- 2: Standing Up, Walking and Going up\down stairs
- 3: Standing
- 4: Walking
- 5: Going Up\Down Stairs
- 6: Walking and Talking with Someone
- 7: Talking while Standing

In [ ]:
# !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00287/Activity%20Recognition%20from%20Single%20Chest-Mounted%20Accelerometer.zip -O ../data/Accelerometer.zip
# ! unzip ../data/Accelerometer.zip -d ./data

In [ ]:
data_path = "../data/Activity Recognition from Single Chest-Mounted Accelerometer/1.csv"
CHANNELS = ['0', '1', '2']
LABELS = [1, 4, 5, 6]
LENGTH = 50
start_ident = 200

In [ ]:
data = pd.read_csv(data_path,
                   names=['0', '1', '2', "labels"],
                   index_col=0,
                   dtype=np.float)

In [ ]:
np.random.seed(0)

dataset = {}
for label in LABELS:
    X = get_class_timeseries(label, data, start_ident)
    X = np.linalg.norm(X, axis=1)[np.newaxis].T
    dataset[label] = slice_timeseries(X, LENGTH, count=45)
    

X = [x for _, ts in dataset.items() for x in ts]
y = [np.repeat(label, ts.shape[0]) for label, ts in dataset.items() for x in ts]
X = np.hstack(X).T
y = np.hstack(y).T

idxs = list(range(X.shape[0]))
shuffle(idxs)

X = X[idxs]
y = y[idxs]

In [ ]:
w = 4
k = 4

input_dim = 2*k
hidden_dim = 5
n_layers = 3
batch_size = 100

In [ ]:
sequences = [split_to_sequence(x, k, w) for x in X[choice(len(X), batch_size)]]

In [ ]:
inp = torch.tensor(sequences, dtype=torch.float).permute(1, 0, 2).cuda()
inp.shape

In [ ]:
lstm = nn.LSTM(input_dim, hidden_dim, n_layers)

In [ ]:
he = torch.randn(n_layers, batch_size, hidden_dim, dtype=torch.float).cuda()
ce = torch.randn(n_layers, batch_size, hidden_dim, dtype=torch.float).cuda()

hd = torch.randn(n_layers, batch_size, input_dim, dtype=torch.float).cuda()
cd = torch.randn(n_layers, batch_size, input_dim, dtype=torch.float).cuda()

In [ ]:
from models import Encoder, Decoder, Sequence2Sequence
enc = Encoder(input_dim, hidden_dim, 1, 1, 3)
dec = Decoder(hidden_dim, input_dim, 1, 1, 3)

In [ ]:
model = Sequence2Sequence(enc, dec)
model.to(torch.device("cuda"))

In [ ]:
loss_fn = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), 0.1)

In [ ]:
N_STEP = 1000

def train(model, batch):
    model.train()
    for step in range(N_STEP):
        out = model(batch, he, ce, hd, cd)
        loss = loss_fn(batch, out)
        
        if step % 100 == 0:
            print("Loss:", loss)
        
        loss.backward()
        optim.step()
        
def valid(model, batch):
    model.eval()
    out = model(batch, he, ce, hd, cd)
    loss = loss_fn(batch, out)
    
    print("Valid loss:", loss)

In [ ]:
model.valid()
train(model, inp)